This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace.

# SETUP

In [1]:
!pip install -U lancedb langchain==0.0.305 openai==0.28.1 torch chromadb

INFO: pip is looking at multiple versions of lancedb to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/29.9 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━

In [2]:
import lancedb
from lancedb.pydantic import LanceModel, vector
import shutil
import pandas as pd
from langchain.llms import OpenAI
import openai
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, NonNegativeInt
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain.document_loaders import CSVLoader
from typing import List
from langchain.prompts import PromptTemplate
from diffusers import DiffusionPipeline, AutoPipelineForText2Image
from diffusers.utils import load_image, make_image_grid
from fastapi.encoders import jsonable_encoder
import PIL


import torch
import sys
import os

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
os.environ["OPENAI_API_KEY"] = "" # Cloud resources to hide when in Github
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [4]:
!echo $OPENAI_API_KEY
!echo $OPENAI_API_BASE

voc-2061880799126677367398467405050402c96.25240448
https://openai.vocareum.com/v1


In [5]:
# Define your OpenAI API key
openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_base = os.environ["OPENAI_API_BASE"]

# Listing Generation

In [6]:
INSTRUCTION = "Generate 10 different and realistic real estate listings from multiple Neighborhoods of Paris."
SAMPLE_LISTING = \
"""
A sample listing is the following as shown below:

"Neighborhood": "Paris 4th District"
"Price (USD)": 773515
"Bedrooms": 3
"Bathrooms": 2
"House Size sqft": 549
"Description": "This south-facing 51 sqm split-level apartment is on the 1st floor of a late 18th century building. Overlooking a peaceful leafy courtyard, it comprises a spacious living room and dining room, and an equipped open-plan kitchen. A bedroom with a shower room and wc is upstairs."
"Neighborhood description": "Rue de Jarente, near Place du March Sainte Catherine in the heart of the historic Marais neighbourhood."

"""

In [7]:
# A real estate listing.
class REListing(BaseModel):
    neighborhood: str = Field(description="Name of the neighborhood")
    price: NonNegativeInt = Field(description="Price of the property in USD")
    bedrooms: NonNegativeInt = Field(description="Number of bedrooms in the property")
    bathrooms: NonNegativeInt = Field(description="Number of bathrooms in the property")
    house_size: NonNegativeInt = Field(description="Size of the property in square feet")
    description: str = Field(description="Description of the property.")
    neighborhood_description: str = Field(description="A description of the neighborhood.")

# A collection of real estate listings
class REListingCollection(BaseModel):
    listings: List[REListing] = Field(description="List of available real estate listings")

# generate parsed output
parser = PydanticOutputParser(pydantic_object=REListingCollection)

In [8]:
prompt = PromptTemplate(
    template="{instruction}\n{sample}\n{format_instructions}\n",
    input_variables=["instruction", "sample"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)
query = prompt.format(instruction = INSTRUCTION, sample = SAMPLE_LISTING)
print(query)

Generate 10 different and realistic real estate listings from multiple Neighborhoods of Paris.

A sample listing is the following as shown below:

"Neighborhood": "Paris 4th District"
"Price (USD)": 773515
"Bedrooms": 3
"Bathrooms": 2
"House Size sqft": 549
"Description": "This south-facing 51 sqm split-level apartment is on the 1st floor of a late 18th century building. Overlooking a peaceful leafy courtyard, it comprises a spacious living room and dining room, and an equipped open-plan kitchen. A bedroom with a shower room and wc is upstairs."
"Neighborhood description": "Rue de Jarente, near Place du March Sainte Catherine in the heart of the historic Marais neighbourhood."


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-f

In [9]:
model_name = "gpt-3.5-turbo"
temperature = 1.0
max_tokens = 3500
llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens = max_tokens)

# get the response
response = llm(query)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [10]:
# create a dataframe from the response
result = parser.parse(response)
df = pd.DataFrame(jsonable_encoder(result.listings))
df.head()

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description
0,Paris 16th District,1550000,4,3,1765,Beautiful 4-bedroom apartment with stunning vi...,"Avenue Foch, near Trocadéro and Bois de Boulogne."
1,Paris 7th District,3200000,2,2,2153,Luxurious 2-bedroom apartment in a historic bu...,"Rue de Grenelle, close to Les Invalides and Mu..."
2,Paris 1st District,980000,1,1,753,Charming 1-bedroom apartment in a classic Haus...,"Rue Saint-Honoré, near Palais Royal and Louvre..."
3,Paris 5th District,620000,2,1,882,Spacious 2-bedroom apartment with a balcony ov...,"Rue Mouffetard, near Jardin des Plantes and Pa..."
4,Paris 10th District,450000,1,1,624,Modern 1-bedroom loft style apartment with ind...,"Canal Saint-Martin, near Place de la République."


In [11]:
df.head(n=10)

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description
0,Paris 16th District,1550000,4,3,1765,Beautiful 4-bedroom apartment with stunning vi...,"Avenue Foch, near Trocadéro and Bois de Boulogne."
1,Paris 7th District,3200000,2,2,2153,Luxurious 2-bedroom apartment in a historic bu...,"Rue de Grenelle, close to Les Invalides and Mu..."
2,Paris 1st District,980000,1,1,753,Charming 1-bedroom apartment in a classic Haus...,"Rue Saint-Honoré, near Palais Royal and Louvre..."
3,Paris 5th District,620000,2,1,882,Spacious 2-bedroom apartment with a balcony ov...,"Rue Mouffetard, near Jardin des Plantes and Pa..."
4,Paris 10th District,450000,1,1,624,Modern 1-bedroom loft style apartment with ind...,"Canal Saint-Martin, near Place de la République."
5,Paris 8th District,2800000,3,2,1981,Elegant 3-bedroom apartment in a prestigious b...,"Avenue des Champs-Élysées, near Arc de Triomph..."
6,Paris 3rd District,780000,2,1,915,Cozy 2-bedroom apartment in a historic buildin...,"Rue de Bretagne, near Musée Picasso and Place ..."
7,Paris 20th District,380000,1,1,517,Bright 1-bedroom apartment with a balcony over...,"Rue des Pyrénées, close to Père Lachaise Cemet..."
8,Paris 12th District,580000,2,1,732,Modern 2-bedroom apartment with a private terr...,"Avenue Daumesnil, near Bois de Vincennes and V..."
9,Paris 6th District,2100000,3,2,1647,Renovated 3-bedroom apartment in a historic bu...,"Boulevard Saint-Germain, near Jardin du Luxemb..."


## Vector Store and Embeddings



In [ ]:
## ### Load them from CSV

In [41]:
def load_listings_from_dataframe(df: pd.DataFrame) -> REListingCollection:
    listings = []
    for _, row in df.iterrows():
        listing = REListing(
            neighborhood=row["neighborhood"],
            price=int(row["price"]),
            bedrooms=int(row["bedrooms"]),
            bathrooms=int(row["bathrooms"]),
            house_size=int(row["house_size"]),
            description=row["description"],
            neighborhood_description=row["neighborhood_description"],
        )
        listings.append(listing)
    return REListingCollection(listings=listings)

# Example usage
#df = pd.read_csv("RElistings.csv")
csv_collection = load_listings_from_dataframe(df)
# print(csv_collection)


In [42]:
print(csv_collection)

listings=[REListing(neighborhood='Paris 16th District', price=1550000, bedrooms=4, bathrooms=3, house_size=1765, description='Beautiful 4-bedroom apartment with stunning views of the Eiffel Tower. Renovated with high-end finishes, spacious living areas, and a modern kitchen. The building offers concierge service and is located in a prestigious area.', neighborhood_description='Avenue Foch, near Trocadéro and Bois de Boulogne.'), REListing(neighborhood='Paris 7th District', price=3200000, bedrooms=2, bathrooms=2, house_size=2153, description='Luxurious 2-bedroom apartment in a historic building with beautiful architecture. Features high ceilings, large windows, and a fully equipped kitchen. Located near the Seine River and famous landmarks.', neighborhood_description="Rue de Grenelle, close to Les Invalides and Musée d'Orsay."), REListing(neighborhood='Paris 1st District', price=980000, bedrooms=1, bathrooms=1, house_size=753, description='Charming 1-bedroom apartment in a classic Hauss

In [55]:
## Generate text to embed

def build_text_for_embedding(listing: REListing) -> str:
    return (
        f"Neighborhood: {listing.neighborhood}\n"
        f"Price: {listing.price}\n"
        f"Bedrooms: {listing.bedrooms}\n"
        f"Bathrooms: {listing.bathrooms}\n"
        f"Description: {listing.description}\n\n"
        f"Neighborhood Description: {listing.neighborhood_description}"
    )

build_text_for_embedding(csv_collection.listings[0])

'Neighborhood: Paris 16th District\nPrice: 1550000\nBedrooms: 4\nBathrooms: 3\nDescription: Beautiful 4-bedroom apartment with stunning views of the Eiffel Tower. Renovated with high-end finishes, spacious living areas, and a modern kitchen. The building offers concierge service and is located in a prestigious area.\n\nNeighborhood Description: Avenue Foch, near Trocadéro and Bois de Boulogne.'

In [56]:
## Generate Embeddings

import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_embedding(text: str) -> list[float]:
    """Use OpenAI to generate an embedding vector."""
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    # "data[0].embedding" is the vector
    return response["data"][0]["embedding"]

#emb = generate_embedding(build_text_for_embedding(csv_collection.listings[0]))
#print(emb)
#print(len(emb))
data = []
for listing in csv_collection.listings:
    embed = generate_embedding(build_text_for_embedding(listing))
    print(build_text_for_embedding(listing))
    print("Embeddings:",embed)
    print(len(embed))
    #print("Neighborhood:",listing.neighborhood)
    #print("Price (USD):",listing.price)
    #print("Bedrooms:",listing.bedrooms)
    #print("Bathrooms:",listing.bathrooms)
    #print("Description:",listing.description)
    #print("Neighborhood Description:",listing.neighborhood_description)
    data.append(RE_listing(vector=embed,Price=listing.price,Neighborhood=listing.neighborhood,Bedrooms=listing.bedrooms,Bathrooms=listing.bathrooms, Description=listing.description,Neighborhood_Description=listing.neighborhood_description))
    print("")

Neighborhood: Paris 16th District
Price: 1550000
Bedrooms: 4
Bathrooms: 3
Description: Beautiful 4-bedroom apartment with stunning views of the Eiffel Tower. Renovated with high-end finishes, spacious living areas, and a modern kitchen. The building offers concierge service and is located in a prestigious area.

Neighborhood Description: Avenue Foch, near Trocadéro and Bois de Boulogne.
Embeddings: [-0.007942035794258118, 0.013142101466655731, -0.016967955976724625, -0.004761350341141224, 0.02579387277364731, 0.010451744310557842, -0.007767839822918177, -0.004935545846819878, -0.04299408942461014, -0.00034335110103711486, -0.0016645371215417981, 0.017780868336558342, -0.011213044635951519, -0.02849068120121956, -0.02807777374982834, -0.00826461985707283, 0.015122771263122559, 0.0036194005515426397, -0.001493567251600325, -0.0066581484861671925, -0.03618109971284866, 0.013058229349553585, 0.0029726182110607624, -0.01044529303908348, -0.005051676649600267, -0.01347113773226738, 0.0204131

In [58]:
data

[RE_listing(vector=FixedSizeList(dim=1536), Neighborhood='Paris 16th District', Price=1550000, Bedrooms=4, Bathrooms=3, Description='Beautiful 4-bedroom apartment with stunning views of the Eiffel Tower. Renovated with high-end finishes, spacious living areas, and a modern kitchen. The building offers concierge service and is located in a prestigious area.', Neighborhood_Description='Avenue Foch, near Trocadéro and Bois de Boulogne.'),
 RE_listing(vector=FixedSizeList(dim=1536), Neighborhood='Paris 7th District', Price=3200000, Bedrooms=2, Bathrooms=2, Description='Luxurious 2-bedroom apartment in a historic building with beautiful architecture. Features high ceilings, large windows, and a fully equipped kitchen. Located near the Seine River and famous landmarks.', Neighborhood_Description="Rue de Grenelle, close to Les Invalides and Musée d'Orsay."),
 RE_listing(vector=FixedSizeList(dim=1536), Neighborhood='Paris 1st District', Price=980000, Bedrooms=1, Bathrooms=1, Description='Charm

In [59]:
# use pydantic to make this easier. First let's create a pydantic model with those fields
from lancedb.pydantic import vector, LanceModel

class RE_listing (LanceModel):
    vector: vector(1536)
    Neighborhood: str
    Price: int
    Bedrooms: int
    Bathrooms: int
    Description: str
    Neighborhood_Description: str

In [60]:
# Now connect to a local db at ~/.lancedb and create an empty LanceDB table
import lancedb

db = lancedb.connect("~/.lancedb")
table_name = "RE_listing"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, schema=RE_listing)

In [61]:
# API to insert these two records into the table
table.add([dict(d) for d in data])

In [62]:
# Let's preview the data
table.head(n=10).to_pandas()

,vector,Neighborhood,Price,Bedrooms,Bathrooms,Description,Neighborhood_Description
0,"[-0.007942036, 0.0131421015, -0.016967956, -0....",Paris 16th District,1550000,4,3,Beautiful 4-bedroom apartment with stunning vi...,"Avenue Foch, near Trocadéro and Bois de Boulogne."
1,"[-0.00044101456, 0.024948362, -0.0058404193, 0...",Paris 7th District,3200000,2,2,Luxurious 2-bedroom apartment in a historic bu...,"Rue de Grenelle, close to Les Invalides and Mu..."
2,"[0.00802122, 0.020117946, -0.011181684, -0.008...",Paris 1st District,980000,1,1,Charming 1-bedroom apartment in a classic Haus...,"Rue Saint-Honoré, near Palais Royal and Louvre..."
3,"[-0.0049059046, 0.028612332, 0.0028612332, -0....",Paris 5th District,620000,2,1,Spacious 2-bedroom apartment with a balcony ov...,"Rue Mouffetard, near Jardin des Plantes and Pa..."
4,"[0.0035981499, 0.00054390635, -0.015152137, -0...",Paris 10th District,450000,1,1,Modern 1-bedroom loft style apartment with ind...,"Canal Saint-Martin, near Place de la République."
5,"[-0.0058417898, 0.0235482, -0.012246098, -0.00...",Paris 8th District,2800000,3,2,Elegant 3-bedroom apartment in a prestigious b...,"Avenue des Champs-Élysées, near Arc de Triomph..."
6,"[-0.008428231, 0.026382323, 0.00328309, 0.0077...",Paris 3rd District,780000,2,1,Cozy 2-bedroom apartment in a historic buildin...,"Rue de Bretagne, near Musée Picasso and Place ..."
7,"[0.015685648, 0.019707942, 0.008864669, -0.009...",Paris 20th District,380000,1,1,Bright 1-bedroom apartment with a balcony over...,"Rue des Pyrénées, close to Père Lachaise Cemet..."
8,"[-0.0070338785, 0.026435442, -0.020491686, 0.0...",Paris 12th District,580000,2,1,Modern 2-bedroom apartment with a private terr...,"Avenue Daumesnil, near Bois de Vincennes and V..."
9,"[0.0056012427, 0.024199951, -0.007231576, 0.00...",Paris 6th District,2100000,3,2,Renovated 3-bedroom apartment in a historic bu...,"Boulevard Saint-Germain, near Jardin du Luxemb..."


## Testing Semantic Search of Listings Based on Buyer Preferences

In [63]:
user_query = "Looking for a 1-bedroom apartment with a balcony"
query_embed = generate_embedding(user_query)

# Search for the 3 most similar listings
results = table.search(query_embed).limit(3).to_pandas()

print(results)

                                              vector         Neighborhood  \
0  [0.015685648, 0.019707942, 0.008864669, -0.009...  Paris 20th District   
1  [-0.0049059046, 0.028612332, 0.0028612332, -0....   Paris 5th District   
2  [0.0035981499, 0.00054390635, -0.015152137, -0...  Paris 10th District   

    Price  Bedrooms  Bathrooms  \
0  380000         1          1   
1  620000         2          1   
2  450000         1          1   

                                         Description  \
0  Bright 1-bedroom apartment with a balcony over...   
1  Spacious 2-bedroom apartment with a balcony ov...   
2  Modern 1-bedroom loft style apartment with ind...   

                            Neighborhood_Description  _distance  
0  Rue des Pyrénées, close to Père Lachaise Cemet...   0.320208  
1  Rue Mouffetard, near Jardin des Plantes and Pa...   0.342447  
2   Canal Saint-Martin, near Place de la République.   0.355341  


## Building the User Preference Interface

In [64]:
questions = [
                "How big do you want your house to be?"
                "What are 3 most important things for you in choosing this property?",
                "Which amenities would you like?",
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",
            ]

In [75]:
# helper function
def get_top_listings(preferences: list[str], top_k: int=5) -> list[int]:
    '''
    Returns the ids of the top k listing
    '''
    top_listingsP = []
    top_listingsBed = []
    top_listingsBath = []
    top_listingsSize = []
    top_listingsDesc = []
    top_listingsNeigh = []
    top_listingsNeighDesc = []
    combined_pref = '\n'.join(preferences)

    query_embed = generate_embedding(combined_pref)

    # Search for the 3 most similar listings
    results = table.search(query_embed).limit(top_k).to_pandas()

    # Iterate through the rows of the DataFrame
    for index, row in results.iterrows():
        top_listingsNeigh.append(row['Neighborhood'])
        top_listingsP.append(row['Price'])
        top_listingsBed.append(row['Bedrooms'])
        top_listingsBath.append(row['Bathrooms'])
        top_listingsSize.append(row['Description'])
        top_listingsNeighDesc.append(row['Neighborhood_Description'])

        # already gotten top listings
        if len(top_listingsNeigh) == top_k:
            break

    return top_listingsP, top_listingsBed, top_listingsBath, top_listingsSize, top_listingsNeigh, top_listingsNeighDesc


In [76]:
# hardcode the buyer's preference in ANSWERS

ANSWERS = [
                "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
                "A quiet neighborhood, good local schools, and convenient shopping options.",
                "A backyard for gardening, a garage, and a modern",
                "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
                "access to urban amenities like restaurants and theaters."
          ]
resP, resBed, resBath, resSize, resNeig, resNeigDesc = get_top_listings(ANSWERS)

# check top listing
#print(texts[res[0]])
#images[res[0]]

In [77]:
print(resP[0])
print(resBed[0])
print(resBath[0])
print(resSize[0])
print(resNeig[0])
print(resNeigDesc[0])

780000
2
1
Cozy 2-bedroom apartment in a historic building with exposed beams and stone walls. Renovated with modern comforts, stylish decor, and an open kitchen. Surrounded by trendy boutiques and cafes.
Paris 3rd District
Rue de Bretagne, near Musée Picasso and Place des Vosges.
